In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import sqlite3
from sqlite3 import Error

resultDir= "TODO configure me"
repeatCount=5

def getScalarValueFromDB(dbConnexion, query):
    cursor=dbConnexion.cursor()
    cursor.execute(query)
    return cursor.fetchone()[0]

def getDBPath(scenario,offeredLoadRate,disruptiveTrafficLevel,repeatID):
    return resultDir+"/"+scenario+"/"+scenario+"-OfferedLoadRate="+str(offeredLoadRate)+",disruptiveTrafficLevel="+str(disruptiveTrafficLevel)+",nbrTrs=30,bitrate=6,packetLength=200,repeatID="+str(repeatID)+"-#0.sca"

    
def getThroughputMeanAndStd(scenario,OfferedLoadRate,disruptiveTrafficLevel):
    def getThroughput(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID):
        dbPath=getDBPath(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID)
        dbConn = sqlite3.connect(dbPath)

        tmpList=[]
        for moduleIndex in range(20,30):
            query = "SELECT scalarValue FROM scalar WHERE moduleName='SimpleNetwork2.node["+str(moduleIndex)+"].applLayer[0]' AND scalarName='throughput:mean'"
            scalar = getScalarValueFromDB(dbConn, query) 
            if scalar is not None:
                tmpList.append(scalar)

        return np.sum(tmpList)
    
    values=[]
    for repeatID in range(repeatCount):
        values.append(getThroughput(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID))
    return [np.mean(values)*0.000001,np.std(values)*0.000001]  # times *0.000001 to convert from bps to mbps     


def getDelayMeanAndStd(scenario,OfferedLoadRate,disruptiveTrafficLevel):
    def getDelay(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID):
        dbPath=getDBPath(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID)
        #print(dbPath)
        dbConn = sqlite3.connect(dbPath)

        tmpList=[]
        for moduleIndex in range(20,30):
            query = "SELECT scalarValue FROM scalar WHERE moduleName='SimpleNetwork2.node["+str(moduleIndex)+"].applLayer[0]' AND scalarName='delay:mean'"
            scalar = getScalarValueFromDB(dbConn, query) 
            if scalar is not None:
                tmpList.append(scalar)

        return np.mean(tmpList)
    
    values=[]
    for repeatID in range(repeatCount):
        values.append(getDelay(scenario,OfferedLoadRate,disruptiveTrafficLevel,repeatID))
    return [np.mean(values*1000),np.std(values)]  # times *1000 to convert from secand to millisecond


def getReliabilityMeanAndStd(scenario,offeredLoadRate,disruptiveTrafficLevel):
    def getReliability(scenario,offeredLoadRate,disruptiveTrafficLevel,repeatID):
        dbPath=getDBPath(scenario,offeredLoadRate,disruptiveTrafficLevel,repeatID)
        dbConn = sqlite3.connect(dbPath)

        tmpList=[]
        for moduleIndex in range(10,20):
            sentPkQuery = "SELECT scalarValue FROM scalar WHERE moduleName='SimpleNetwork2.node["+str(moduleIndex)+"].applLayer[0]' AND scalarName='sentPk:count'"
            rcvdPkQuery = "SELECT scalarValue FROM scalar WHERE moduleName='SimpleNetwork2.node["+str(10+moduleIndex)+"].applLayer[0]' AND scalarName='rcvdPk:count'"
            sentPk = getScalarValueFromDB(dbConn, sentPkQuery)
            rcvdPk = getScalarValueFromDB(dbConn, rcvdPkQuery)
            
            if rcvdPk is not None:
                assert sentPk > 0 
                tmpList.append(rcvdPk/sentPk)

        return np.mean(tmpList)
    
    values=[]
    for repeatID in range(repeatCount):
        values.append(getReliability(scenario,offeredLoadRate,disruptiveTrafficLevel,repeatID))
    return [np.mean(values)*100,np.std(values)*100]  # times *100 to convert to %


def plotResult(scenario,baseline,resultTypeCode):
    
    for offeredLoadRate in [30]:
        
        ylabelTitle=""
        scenarioData=[]
        scenarioYerr=[]
        baselineData=[]
        baselineYerr=[] 
        
        disruptiveTrafficLevels =[]
        
        for disruptiveTrafficLevel in range(20,100,20):
            disruptiveTrafficLevels.append(disruptiveTrafficLevel)
            
            if resultTypeCode == 0 :
                ylabelTitle="Delay (ms)"
                tmp = getDelayMeanAndStd(scenario,offeredLoadRate,disruptiveTrafficLevel)
                scenarioData.append(tmp[0])
                scenarioYerr.append(tmp[1])
                tmp = getDelayMeanAndStd(baseline,offeredLoadRate,disruptiveTrafficLevel)
                baselineData.append(tmp[0])
                baselineYerr.append(tmp[1])  
            elif resultTypeCode == 1 :
                ylabelTitle="Throughput (Mbps)"
                tmp = getThroughputMeanAndStd(scenario,offeredLoadRate,disruptiveTrafficLevel)
                scenarioData.append(tmp[0])
                scenarioYerr.append(tmp[1])
                tmp = getThroughputMeanAndStd(baseline,offeredLoadRate,disruptiveTrafficLevel)
                baselineData.append(tmp[0])
                baselineYerr.append(tmp[1])
                
            elif resultTypeCode == 2 :
                ylabelTitle="Data delivery rate (%)"
                tmp = getReliabilityMeanAndStd(scenario,offeredLoadRate,disruptiveTrafficLevel)
                scenarioData.append(tmp[0])
                scenarioYerr.append(tmp[1])
                tmp = getReliabilityMeanAndStd(baseline,offeredLoadRate,disruptiveTrafficLevel)
                baselineData.append(tmp[0])
                baselineYerr.append(tmp[1])
        
        fig,ax = plt.subplots()

        plt.ylabel(ylabelTitle) 
        plt.title("["+str(offeredLoadRate)+"%"" decision traffic load]")
        

        ax.errorbar(disruptiveTrafficLevels,scenarioData,yerr=scenarioYerr,label="DICART-"+scenario)
        ax.errorbar(disruptiveTrafficLevels,baselineData,yerr=baselineYerr,label="Baseline-"+baseline)

        lgd = ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.16),ncol=3,  shadow=True)

        ax.grid()
        fig.savefig(resultDir+"/"+scenario+"-"+str(offeredLoadRate)+"-"+str(resultTypeCode),bbox_extra_artists=(lgd,), bbox_inches='tight')
        plt.show() 


In [ ]:
scenario="equalWeight"
baseline="random"

for resultTypeCode in range(0,3):  
    plotResult(scenario,baseline,resultTypeCode)


In [ ]:
scenario="delayWeighted"
baseline="naiveDelayBasedDecision"

for resultTypeCode in range(0,3):  
    plotResult(scenario,baseline,resultTypeCode)

In [ ]:
scenario="throughputWeighted"
baseline="naiveThroughputBasedDecision"

for resultTypeCode in range(0,3):  
    plotResult(scenario,baseline,resultTypeCode)

In [ ]:
scenario="reliabilityWeighted"
baseline="naiveReliabilityBasedDecision"

for resultTypeCode in range(0,3):  
    plotResult(scenario,baseline,resultTypeCode)